## 0. 필요 라이브러리 import

In [1]:
!pip install google-cloud-storage

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.9/174.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 23.5 MB/s eta 0:00:00


In [2]:
from google.oauth2 import service_account
from google.cloud import storage

## 1. GCP 권한 인증

In [3]:
## 권한 인증

JSON_KEY_PATH = "sa_key/sprintda05_DE_key.json"

credentials = service_account.Credentials.from_service_account_file(JSON_KEY_PATH)

In [4]:
## client 생성

project_id = "sprintda05-hyunsoo"
location = "asia-northeast3"

client = storage.Client(
    project=project_id,
    credentials=credentials
    )

## 2. Parquet 파일 다루기
- 데이터를 객체 저장소에 저장할 때는, 용량 효율성, 빅데이터 도구들과의 호환성, 파티셔닝 등의 이점 때문에 CSV나 JSON 같은 포맷보다 Parquet 포맷을 사용하는 경우가 많습니다. (대부분 Parquet 사용)

In [1]:
## parquet 파일 읽기
import pandas as pd

df = pd.read_parquet("dataset/member.parquet")

df.head()

,idx,sex,status,grade
0,100,남,유료회원,초1학년
1,1000,여,유료회원,초5학년
2,10000,여,유료회원,초6학년
3,100007,남,유료회원,초4학년
4,10001,남,유료회원,초2학년


In [2]:
## parquet 파일 쓰기 (데이터프레임 -> Parquet 파일로 저장)

df.to_parquet(
    path='dataset/member_parquet.parquet',
    index=False,
    engine='pyarrow',
    compression='gzip'
)

## 3. Bucket 관련 작업

In [5]:
## 1) 버킷 목록 가져오기

for bk in client.list_buckets():
    print(bk.name)

sprintda05-airflow-hyunsoo-bucket
sprintda05-hyunsoo-bucket


In [ ]:
## 2) 버킷 생성

NEW_BUCKET_NAME = "새로운 버킷 이름"

client.create_bucket(
    bucket_or_name=NEW_BUCKET_NAME,
    project=project_id,
    location=location
    )

In [ ]:
## 3) 버킷 정보 조회

bucket = client.get_bucket(NEW_BUCKET_NAME)
vars(bucket)

In [ ]:
## 4) 버킷 삭제

bucket = client.get_bucket(NEW_BUCKET_NAME)
bucket.delete()

## 4. 객체 관련 작업

In [ ]:
## 1) 객체 업로드 - 파일

bucket = client.get_bucket(NEW_BUCKET_NAME)

blob = bucket.blob(blob_name="로컬 파일 경로")
blob.upload_from_filename("저장될 객체 파일명")

In [ ]:
## 2) 객체 업로드 - DataFrame

import pandas as pd

df = pd.read_csv('names.csv')

bucket = client.get_bucket(NEW_BUCKET_NAME)

blob = bucket.blob(blob_name="my_dir/names_df.csv")

blob.upload_from_string(
    data=df.to_csv(index=False),
    content_type='text/csv'
    )

In [ ]:
## 3) 객체 다운로드

bucket = client.get_bucket(NEW_BUCKET_NAME)

blob = bucket.blob(blob_name="my_dir/tips.csv")

blob.download_to_filename("tips.csv")

In [ ]:
## 4) 폴더 내 객체 목록 가져오기

blobs = client.list_blobs(
    bucket_or_name=NEW_BUCKET_NAME,
    prefix="my_dir/"
    )

for b in blobs:
    print(vars(b))

In [ ]:
## 5) 객체 유무 조회

bucket = client.get_bucket(NEW_BUCKET_NAME)

blob = bucket.blob(blob_name="my_dir/tips.csv")

blob.exists()

In [ ]:
## 6) 객체 삭제

bucket = client.get_bucket(NEW_BUCKET_NAME)

blob = bucket.blob(blob_name="my_dir/tips.csv")

blob.delete()

## 5. Pandas로 GCS 데이터 읽기
- GCS를 작업자의 로컬 디렉토리 처럼 활용하여 데이터를 조회할 수 있는 매우 편리한 기능!

1. 로컬 작업 디렉토리에 위치한 파일 (read_csv, read_parquet)
2. 데이터베이스에 저장된 테이블 (read_sql)
3. 빅쿼리에 저장된 테이블 (read_gbq)
4. 클라우드 객체 저장소(GCS)에 저장된 파일 (read_csv, read_parquet)

In [3]:
df = pd.read_parquet("dataset/member.parquet")

df.head()

,idx,sex,status,grade
0,100,남,유료회원,초1학년
1,1000,여,유료회원,초5학년
2,10000,여,유료회원,초6학년
3,100007,남,유료회원,초4학년
4,10001,남,유료회원,초2학년


In [8]:
## 서비스 계정 JSON 키
JSON_KEY_PATH = "sa_key/sprintda05_DE_key.json"

pd.read_csv(
    filepath_or_buffer="gs://sprintda05-hyunsoo-bucket/edu_dataset/point_his.csv",
    storage_options={"token" : JSON_KEY_PATH}
)

,idx,proc_ym,proc_ymd,point
0,87376,202304,20230401,1000
1,87599,202304,20230401,1000
2,87682,202304,20230401,1000
3,87555,202304,20230401,1000
4,87569,202304,20230401,1000
...,...,...,...,...
92074,97018,202306,20230630,5000
92075,97035,202306,20230630,5000
92076,96869,202306,20230630,5000
92077,96751,202306,20230630,5000


In [7]:
pd.read_parquet(
    path="gs://sprintda05-hyunsoo-bucket/bigquery_data/regdate.parquet",
    storage_options={"token" : JSON_KEY_PATH}
)

,idx,regdate
0,1,20221206
1,2,20221206
2,3,20221206
3,4,20221206
4,5,20221206
...,...,...
112563,112565,20231118
112564,112566,20231118
112565,112567,20231118
112566,112568,20231118


## 6. (빅)데이터 저장시 Parquet 파일을 사용하는 이유

- 용량! -> CSV에 비해 훨씬 작은 용량을 차지
- 비용! -> 같은 데이터 저장시 비용이 덜 발생
- metedata 같이 저장 -> 저장/읽을 때 더 빠르고 편리
- 빅데이터 관련 도구들과 호환 -> kafka, spark, flink, trino, hive, iceberg 등 빅데이터 도구들과 높은 호환성
- 파티셔닝 기능 -> 객체 저장소에 저장될 때 자동으로 파티셔닝이 가능!

In [ ]:
# 데이터가 굉장히 커지면
    - DB나 bigquery에 테이블로 저장
        - 높은 비용이 발생!
    
    - 객체 스토리지에 파일로 저장(parquet)
        - 비교적 적은 비용
        - 다른 도구들과 호환성이 뛰어남!

In [ ]:
# 데이터 저장시 바람직한 저장 방법이 X
data/
    20250501.parquet
    20250502.parquet
    20250503.parquet
    20250504.parquet

In [ ]:
# 데이터 저장시 올바른 저장 방법!
data/
    - yyyy=2023/
        - mm=04/
            - dd=01
                - data.parquet
            - dd=02
                - data.parquet
            - dd=03
                - data.parquet

In [ ]:
JSON_KEY_PATH = "sa_key/sprintda05_DE_key.json"

## 상위 폴더 디렉토리를 read하는 경우 에러 발생!!

pd.read_csv(
    filepath_or_buffer="gs://sprintda05-hyunsoo-bucket/gcs_part_csv/",
    storage_options={"token" : JSON_KEY_PATH}
)

In [15]:
## 상위 폴더 디렉토리를 read하는 경우 정상적으로 읽힘!

pd.read_parquet(
    path="gs://sprintda05-hyunsoo-bucket/gcp_part_parquet",
    storage_options={"token" : JSON_KEY_PATH}
)

,idx,proc_ym,proc_ymd,pointnm,yyyy,mm,dd
0,75178,202304,2023-04-01,한글 스피치,2023,4,1
1,7287,202304,2023-04-01,중학 1학년,2023,4,1
2,5547,202304,2023-04-01,중학 1학년,2023,4,1
3,80092,202304,2023-04-01,중학 1학년,2023,4,1
4,67940,202304,2023-04-01,중학 2학년,2023,4,1
...,...,...,...,...,...,...,...
44417,12257,202304,2023-04-30,밀크T 지오그래픽,2023,4,30
44418,12257,202304,2023-04-30,밀크T 지오그래픽,2023,4,30
44419,12257,202304,2023-04-30,밀크T 지오그래픽,2023,4,30
44420,66583,202304,2023-04-30,밀크T 지오그래픽,2023,4,30


In [1]:
from pprint import pprint

In [5]:
aa = dict(
        name = 'codeit',
        age = 20,
        count = 100,
        count1 = 100,
        count2 = 100,
        count3 = 100,
        )

In [6]:
pprint(aa)

{'age': 20,
 'count': 100,
 'count1': 100,
 'count2': 100,
 'count3': 100,
 'name': 'codeit'}
